# residual network classifier, code modified from [tensorflow/model](https://github.com/tensorflow/models/tree/master/official/resnet)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import resnet_model
import os

df = pd.read_csv('./fashion-mnist_train.csv')
label = df['label']
X = df.iloc[:, 1:]
y = np.asarray(label, np.int32)
X = np.asarray(X, np.float32)

HEIGHT = 28
WIDTH = 28
DEPTH = 1
NUM_CLASSES = 10
_WEIGHT_DECAY = 2e-4
TRAIN_STEPS = 100000
STEPS_PER_EVAL = 2000
RESNET_SIZE = 32

X = X.reshape((X.shape[0], HEIGHT, WIDTH, DEPTH))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = len(X_train)


BATCH_SIZE = 128
_INITIAL_LEARNING_RATE = 0.1 * BATCH_SIZE / 128
_MOMENTUM = 0.9

# We use a weight decay of 0.0002, which performs better than the 0.0001 that
# was originally suggested.
_WEIGHT_DECAY = 2e-4

_BATCHES_PER_EPOCH = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN // BATCH_SIZE



In [2]:
dataset = tf.contrib.data.Dataset.from_tensor_slices((X_train, tf.one_hot(y_train, NUM_CLASSES)))
print(dataset.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset.output_shapes)  # ==> "{'a': (), 'b': (100,)}"

(tf.float32, tf.float32)
(TensorShape([Dimension(28), Dimension(28), Dimension(1)]), TensorShape([Dimension(10)]))


In [3]:
iterator = dataset.batch(10).make_one_shot_iterator()
images, labels = iterator.get_next()

In [4]:
def create_dataset(x, y):
    dataset = tf.contrib.data.Dataset.from_tensor_slices((x, tf.one_hot(y, NUM_CLASSES)))
    return dataset


In [5]:
def train_preprocess_fn(image, label):
    """Preprocess a single training image of layout [height, width, depth]."""
    # Resize the image to add four extra pixels on each side.
    image = tf.image.resize_image_with_crop_or_pad(image, HEIGHT + 8, WIDTH + 8)

    # Randomly crop a [HEIGHT, WIDTH] section of the image.
    image = tf.random_crop(image, [HEIGHT, WIDTH, DEPTH])

    # Randomly flip the image horizontally.
    image = tf.image.random_flip_left_right(image)

    return image, label


def input_fn(mode, batch_size):
    """Input_fn using the contrib.data input pipeline.
    Args:
    mode: Standard names for model modes from tf.estimator.ModeKeys.
    batch_size: The number of samples per batch of input requested.
    Returns:
    A tuple of images and labels.
    """
    # For training repeat forever.
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = create_dataset(X_train, y_train)
        dataset = dataset.repeat()
    else:
        dataset = create_dataset(X_test, y_test)

    # For training, preprocess the image and shuffle.
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.map(train_preprocess_fn, num_threads=1,
                              output_buffer_size=2 * batch_size)

    # Ensure that the capacity is sufficiently large to provide good random
    # shuffling.
    buffer_size = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * 0.4) + 3 * batch_size
    dataset = dataset.shuffle(buffer_size=buffer_size)

    # Subtract off the mean and divide by the variance of the pixels.
    dataset = dataset.map(
        lambda image, label: (tf.image.per_image_standardization(image), label),
        num_threads=1,
        output_buffer_size=2 * batch_size)

    # Batch results by up to batch_size, and then fetch the tuple from the
    # iterator.
    iterator = dataset.batch(batch_size).make_one_shot_iterator()
    images, labels = iterator.get_next()

    return images, labels

In [6]:
def cifar10_model_fn(features, labels, mode):
    """Model function for CIFAR-10."""
    tf.summary.image('images', features, max_outputs=6)

    network = resnet_model.cifar10_resnet_v2_generator(
      RESNET_SIZE, NUM_CLASSES, data_format='channels_last')

    inputs = tf.reshape(features, [-1, HEIGHT, WIDTH, DEPTH])
    logits = network(inputs, mode == tf.estimator.ModeKeys.TRAIN)

    predictions = {
      'classes': tf.argmax(logits, axis=1),
      'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate loss, which includes softmax cross entropy and L2 regularization.
    cross_entropy = tf.losses.softmax_cross_entropy(
      logits=logits, onehot_labels=labels)

    # Create a tensor named cross_entropy for logging purposes.
    tf.identity(cross_entropy, name='cross_entropy')
    tf.summary.scalar('cross_entropy', cross_entropy)

    # Add weight decay to the loss.
    loss = cross_entropy + _WEIGHT_DECAY * tf.add_n(
        [tf.nn.l2_loss(v) for v in tf.trainable_variables()])

    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_or_create_global_step()

        # Multiply the learning rate by 0.1 at 100, 150, and 200 epochs.
        boundaries = [int(_BATCHES_PER_EPOCH * epoch) for epoch in [100, 150, 200]]
        values = [_INITIAL_LEARNING_RATE * decay for decay in [1, 0.1, 0.01, 0.001]]
        learning_rate = tf.train.piecewise_constant(
            tf.cast(global_step, tf.int32), boundaries, values)

        # Create a tensor named learning_rate for logging purposes
        tf.identity(learning_rate, name='learning_rate')
        tf.summary.scalar('learning_rate', learning_rate)

        optimizer = tf.train.MomentumOptimizer(
            learning_rate=learning_rate,
            momentum=_MOMENTUM)

        # Batch norm requires update ops to be added as a dependency to the train_op
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss, global_step)
    else:
        train_op = None

    accuracy = tf.metrics.accuracy(
      tf.argmax(labels, axis=1), predictions['classes'])
    metrics = {'accuracy': accuracy}

    # Create a tensor named train_accuracy for logging purposes
    tf.identity(accuracy[1], name='train_accuracy')
    tf.summary.scalar('train_accuracy', accuracy[1])

    return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=predictions,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=metrics)


In [ ]:
def main():
  # Using the Winograd non-fused algorithms provides a small performance boost.
    os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

    cifar_classifier = tf.estimator.Estimator(
      model_fn=cifar10_model_fn, model_dir='./model_weights/')

    for _ in range(TRAIN_STEPS // STEPS_PER_EVAL):
        tensors_to_log = {
            'learning_rate': 'learning_rate',
            'cross_entropy': 'cross_entropy',
            'train_accuracy': 'train_accuracy'
        }

        logging_hook = tf.train.LoggingTensorHook(
            tensors=tensors_to_log, every_n_iter=100)

        cifar_classifier.train(
            input_fn=lambda: input_fn(tf.estimator.ModeKeys.TRAIN,
                                      batch_size=BATCH_SIZE),
            steps=STEPS_PER_EVAL,
            hooks=[logging_hook])

        # Evaluate the model and print results
        eval_results = cifar_classifier.evaluate(
            input_fn=lambda: input_fn(tf.estimator.ModeKeys.EVAL,
                                      batch_size=BATCH_SIZE))
        print(eval_results)


In [ ]:
main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model_weights/', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model_weights/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into ./model_weights/model.ckpt.
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 2.27645, train_accuracy = 0.179688
INFO:tensorflow:loss = 2.48879, step = 2
INFO:tensorflow:global_step/sec: 17.4345
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.797992, train_accuracy = 0.433594 (5.736 sec)
INFO:tensorflow:loss = 1.01791, step = 102 (5.736 sec)
INFO:tensorflow:global_step/sec: 17.8665
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.599079, train_accuracy = 0.546875 (5.597 sec)
INF

INFO:tensorflow:global_step/sec: 16.8665
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.263687, train_accuracy = 0.901654 (5.929 sec)
INFO:tensorflow:loss = 0.391065, step = 3602 (5.929 sec)
INFO:tensorflow:global_step/sec: 16.882
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.266698, train_accuracy = 0.900608 (5.923 sec)
INFO:tensorflow:loss = 0.392768, step = 3702 (5.924 sec)
INFO:tensorflow:global_step/sec: 16.9828
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.185966, train_accuracy = 0.901727 (5.888 sec)
INFO:tensorflow:loss = 0.310852, step = 3802 (5.888 sec)
INFO:tensorflow:global_step/sec: 16.9288
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.237558, train_accuracy = 0.901953 (5.907 sec)
INFO:tensorflow:loss = 0.361229, step = 3902 (5.907 sec)
INFO:tensorflow:Saving checkpoints for 4001 into ./model_weights/model.ckpt.
INFO:tensorflow:Loss for final step: 0.367593.
INFO:tensorflow:Starting evaluation at 2017-10-12-21:23:13
INFO:tensorflow:Restor

INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.180964, train_accuracy = 0.910807 (6.386 sec)
INFO:tensorflow:loss = 0.288438, step = 7102 (6.386 sec)
INFO:tensorflow:global_step/sec: 15.6137
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.181417, train_accuracy = 0.911659 (6.404 sec)
INFO:tensorflow:loss = 0.288488, step = 7202 (6.404 sec)
INFO:tensorflow:global_step/sec: 15.983
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.226105, train_accuracy = 0.912388 (6.257 sec)
INFO:tensorflow:loss = 0.332953, step = 7302 (6.257 sec)
INFO:tensorflow:global_step/sec: 16.1212
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.268859, train_accuracy = 0.911458 (6.203 sec)
INFO:tensorflow:loss = 0.376266, step = 7402 (6.203 sec)
INFO:tensorflow:global_step/sec: 16.1522
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.204033, train_accuracy = 0.911621 (6.191 sec)
INFO:tensorflow:loss = 0.311105, step = 7502 (6.191 sec)
INFO:tensorflow:global_step/sec: 16.1571
INFO:t

INFO:tensorflow:loss = 0.259264, step = 10602 (5.944 sec)
INFO:tensorflow:global_step/sec: 16.8063
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.200431, train_accuracy = 0.913086 (5.952 sec)
INFO:tensorflow:loss = 0.307664, step = 10702 (5.951 sec)
INFO:tensorflow:global_step/sec: 16.937
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.237894, train_accuracy = 0.907986 (5.903 sec)
INFO:tensorflow:loss = 0.344824, step = 10802 (5.901 sec)
INFO:tensorflow:global_step/sec: 16.7609
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.146558, train_accuracy = 0.911719 (5.966 sec)
INFO:tensorflow:loss = 0.253459, step = 10902 (5.966 sec)
INFO:tensorflow:global_step/sec: 16.8467
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.263927, train_accuracy = 0.910511 (5.936 sec)
INFO:tensorflow:loss = 0.371118, step = 11002 (5.936 sec)
INFO:tensorflow:global_step/sec: 16.393
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.175954, train_accuracy = 0.913411 (6.100 sec)
IN

INFO:tensorflow:loss = 0.32407, step = 14102 (6.086 sec)
INFO:tensorflow:global_step/sec: 17.2432
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.228509, train_accuracy = 0.924479 (5.799 sec)
INFO:tensorflow:loss = 0.338146, step = 14202 (5.799 sec)
INFO:tensorflow:global_step/sec: 16.9275
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.235315, train_accuracy = 0.919922 (5.908 sec)
INFO:tensorflow:loss = 0.344792, step = 14302 (5.908 sec)
INFO:tensorflow:global_step/sec: 16.7871
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.20565, train_accuracy = 0.923437 (5.957 sec)
INFO:tensorflow:loss = 0.314909, step = 14402 (5.957 sec)
INFO:tensorflow:global_step/sec: 16.9067
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.210527, train_accuracy = 0.917969 (5.915 sec)
INFO:tensorflow:loss = 0.319994, step = 14502 (5.915 sec)
INFO:tensorflow:global_step/sec: 16.894
INFO:tensorflow:learning_rate = 0.1, cross_entropy = 0.104001, train_accuracy = 0.925223 (5.919 sec)
INF